In [ ]:
from google.colab import files
uploaded = files.upload()


Saving diabetes_binary_health_indicators_BRFSS2015.csv to diabetes_binary_health_indicators_BRFSS2015.csv


In [ ]:
!pip install streamlit==1.46.0 bokeh==2.4.3 numpy==1.26.4 pyngrok

In [13]:
%%writefile app.py
import streamlit as st
import pandas as pd
import altair as alt
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

# Load data
df = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")

# Preprocess
df["Diabetes"] = df["Diabetes_binary"].map({0: "Non-Diabetes", 1: "Diabetes"})
df["Sex_Label"] = df["Sex"].map({0: "Perempuan", 1: "Laki-laki"})
age_labels = {
    1: '18-24', 2: '25-29', 3: '30-34', 4: '35-39', 5: '40-44',
    6: '45-49', 7: '50-54', 8: '55-59', 9: '60-64', 10: '65-69',
    11: '70-74', 12: '75-79', 13: '80+'
}
df["Age_Label"] = df["Age"].map(age_labels)
df["Age_Group"] = df["Age"]

# Page setup
st.set_page_config(page_title="Diabetes Health Indicators Dashboard", layout="wide")

# Styles
st.markdown("""
<style>
    .stApp {
        background: linear-gradient(135deg, #ffe4e1 0%, #ffe8ff 33%, #d8f9ff 66%, #e3ffe7 100%);
    }
    .block-container {
        padding: 2rem;
    }
    .metric-box {
        text-align: center;
        background-color: rgba(255, 255, 255, 0.95);
        padding: 30px 20px;
        border-radius: 15px;
        box-shadow: 2px 2px 10px rgba(0,0,0,0.1);
    }
</style>
""", unsafe_allow_html=True)

# Header
st.markdown("""
<h1 style='text-align: center; color: #FFFFFF; text-shadow: 2px 2px 5px rgba(0,0,0,0.8)'>🩺 Diabetes Health Indicators Dashboard</h1>
<h4 style='text-align: center; color: #FFEE99; text-shadow: 1px 1px 3px rgba(0,0,0,0.5)'>Interactive Analysis of BRFSS 2015 Health Data</h4>
""", unsafe_allow_html=True)


# Filter Options
st.markdown("### 🎛️ Filter Options")
with st.container():
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        age_range = st.selectbox("Age Range", ["All"] + list(age_labels.values()))
    with col2:
        gender = st.selectbox("Gender", ["All", "Laki-laki", "Perempuan"])
    with col3:
        bmi_slider = st.slider("BMI Range", 10, 50, 25)
    with col4:
        x_axis = st.selectbox("X-Axis", ["Age", "BMI", "MentHlth", "PhysHlth"])
    with col5:
        y_axis = st.selectbox("Y-Axis", ["BMI", "MentHlth", "PhysHlth", "Diabetes_binary"])
    if st.button("🔄 Reset Filter"):
        st.experimental_rerun()

# Filter data
filtered_df = df[df["BMI"] >= bmi_slider]
if age_range != "All":
    selected_key = [k for k, v in age_labels.items() if v == age_range][0]
    filtered_df = filtered_df[filtered_df["Age"] == selected_key]
if gender != "All":
    filtered_df = filtered_df[filtered_df["Sex_Label"] == gender]

# Summary boxes
st.markdown("### 📊 Summary")
col1, col2, col3, col4 = st.columns(4)
col1.markdown(f"<div class='metric-box'><h2>{len(filtered_df):,}</h2><p>Total Records</p></div>", unsafe_allow_html=True)
col2.markdown(f"<div class='metric-box'><h2>{(filtered_df['Diabetes_binary'].mean()*100):.1f}%</h2><p>Diabetes Rate (%)</p></div>", unsafe_allow_html=True)
col3.markdown(f"<div class='metric-box'><h2>{filtered_df['BMI'].mean():.1f}</h2><p>Average BMI</p></div>", unsafe_allow_html=True)
col4.markdown(f"<div class='metric-box'><h2>{(filtered_df['BMI'] >= 30).sum():,}</h2><p>High Risk Individuals</p></div>", unsafe_allow_html=True)

st.markdown("<br>", unsafe_allow_html=True)

# Scatterplots
brush = alt.selection_interval()
scatter1 = alt.Chart(filtered_df).mark_circle(size=60).encode(
    x=alt.X(x_axis, title=x_axis),
    y=alt.Y(y_axis, title=y_axis),
    color=alt.Color("Diabetes", scale=alt.Scale(range=["#89CFF0", "#FFB347"])),
    tooltip=["Diabetes", "BMI", "MentHlth", "PhysHlth"]
).add_selection(brush).properties(
    width=600, height=400, title="Scatterplot with Brushing"
).interactive()

scatter2 = alt.Chart(filtered_df).mark_circle(size=60).encode(
    x=alt.X("BMI", title="BMI"),
    y=alt.Y("MentHlth", title="Mental Health"),
    color=alt.Color("Diabetes", scale=alt.Scale(range=["#89CFF0", "#FFB347"])),
    tooltip=["Diabetes", "BMI", "MentHlth"]
).transform_filter(brush).properties(
    width=600, height=400, title="Linked Scatter (BMI vs Mental Health)"
)

st.altair_chart(scatter1 & scatter2)

# Anotasi
highlight = filtered_df[filtered_df["BMI"] > 45]
if not highlight.empty:
    st.markdown("#### ⚠️ BMI Ekstrem Terdeteksi")
    st.dataframe(highlight[["BMI", "Age_Label", "Sex_Label", "Diabetes"]])

st.markdown("<br>", unsafe_allow_html=True)

# Bar chart
st.markdown("### 📈 Distribution by Categories")
diabetes_counts = filtered_df["Diabetes"].value_counts()
categories = diabetes_counts.index.tolist()
counts = diabetes_counts.values.tolist()
colors = ["#89CFF0" if label == "Non-Diabetes" else "#FFB347" for label in categories]

source = ColumnDataSource(data=dict(x=categories, y=counts, color=colors))
p = figure(x_range=categories, height=300, title="Diabetes Distribution",
           toolbar_location=None, tools="")
p.vbar(x='x', top='y', width=0.6, source=source, fill_color='color')
p.xgrid.grid_line_color = None
p.y_range.start = 0
st.bokeh_chart(p, use_container_width=True)

# Footer
st.markdown("""
<hr>
<div style='text-align: center; font-size: 12px; color: #FFFFFF;'>
    Kelompok 15 - Awanda Puspa Larasati | Purinda Kyla Larissa | Rayhan Dhika Setiawan | Final Project Interactive Visualization
</div>
""", unsafe_allow_html=True)


Overwriting app.py


In [11]:
#ini kalau jalanin streamlit lebih dr 3x n gabisa jalanin ini
!pkill streamlit
!pkill ngrok

In [14]:
from pyngrok import ngrok
import subprocess
import time

# Jalanin streamlit
process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(5)

url = ngrok.connect(8501)
print(f"👉 Aplikasi kamu bisa diakses di: {url.public_url}")

👉 Aplikasi kamu bisa diakses di: https://7a4e-35-224-122-60.ngrok-free.app
